# From Last Time

In [2]:
def Alignment(b,v,w,i,j):
    if ((i == 0) and (j == 0)):
        return ['','']
    if (b[i,j] == 3):
        result = Alignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = Alignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return resul
    if (b[i,j] == 1):
        result = Alignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result

In [3]:
def getMax(s):
    N, M = s.shape
    maxValue = -N*M
    maxIJ = (-1, -1)
    for i in range(N):
        for j in range(M):
            if (s[i, j] > maxValue):
                maxValue = s[i, j]
                maxIJ= (i, j)
    return maxIJ

def LocalAlignment(b,v,w):
    i, j = getMax(s)
    if (b[i,j] == 0):
        return['', '']
    if (b[i,j] == 3):
        result = LocalAlignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = LocalAlignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = LocalAlignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result

## Global Alignment using a scoring matrix

In [3]:
import numpy

def GlobalAlign(v, w, scorematrix, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(0,len(v)+1):
        for j in range(0,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = s[i-1,j] + indel
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = s[i,j-1] + indel
                b[i,j] = 2
                continue
            score = s[i-1,j-1] + scorematrix[v[i-1],w[j-1]]
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            else:
                b[i,j] = 3
    return (s, b)

match = {('A','A'):  1, ('A','C'): -2, ('A','G'): -1, ('A','T'): -2,
         ('C','A'): -2, ('C','C'):  1, ('C','G'): -2, ('C','T'): -1,
         ('G','A'): -1, ('G','C'): -2, ('G','G'):  1, ('G','T'): -2,
         ('T','A'): -2, ('T','C'): -1, ('T','G'): -2, ('T','T'):  1}

v = "TTCCGAGCGTTA"
w = "TTTCAGGTTA"

s, b = GlobalAlign(v,w,match,-3)
print("Best score =", s[-1,-1])
align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)
print("v =", align[0])
print("w =", align[1])

Best score = 2
v = TTCCGAGCGTTA
w = TTTC_AG_GTTA


## Local Alignment

In [4]:
import numpy

def LocalAlign(v, w, scorematrix, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = max(s[i-1,j] + indel, 0)
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = max(s[i,j-1] + indel, 0)
                b[i,j] = 2
                continue
            score = s[i-1,j-1] + scorematrix[v[i-1],w[j-1]]
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score, 0)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            elif (s[i,j] == score):
                b[i,j] = 3
            else:
                b[i,j] = 0
    return (s, b)

match = {('A','A'):  5, ('A','C'): -4, ('A','G'): -4, ('A','T'): -4,
         ('C','A'): -4, ('C','C'):  5, ('C','G'): -4, ('C','T'): -4,
         ('G','A'): -4, ('G','C'): -4, ('G','G'):  5, ('G','T'): -4,
         ('T','A'): -4, ('T','C'): -4, ('T','G'): -4, ('T','T'):  5}

v = "GCTGGAAGGCAT"
w = "GCAGAGCACT"

s, b = LocalAlign(v,w,match,-7)
print(s)
print()
print(b)

[[ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  5  0  5  0  0  0  0]
 [ 0  0 10  3  0  1  0 10  3  5  0]
 [ 0  0  3  6  0  0  0  3  6  0 10]
 [ 0  5  0  0 11  4  5  0  0  2  3]
 [ 0  5  1  0  5  7  9  2  0  0  0]
 [ 0  0  1  6  0 10  3  5  7  0  0]
 [ 0  0  0  6  2  5  6  0 10  3  0]
 [ 0  5  0  0 11  4 10  3  3  6  0]
 [ 0  5  1  0  5  7  9  6  0  0  2]
 [ 0  0 10  3  0  1  3 14  7  5  0]
 [ 0  0  3 15  8  5  0  7 19 12  5]
 [ 0  0  0  8 11  4  1  0 12 15 17]]

[[0 0 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 3 0 3 0 0 0 0]
 [0 0 3 2 0 3 0 3 2 3 0]
 [0 0 1 3 0 0 0 1 3 0 3]
 [0 3 0 0 3 2 3 0 0 3 1]
 [0 3 3 0 3 3 3 2 0 0 0]
 [0 0 3 3 0 3 2 3 3 2 0]
 [0 0 0 3 3 3 3 0 3 2 0]
 [0 3 0 0 3 2 3 2 1 3 0]
 [0 3 3 0 3 3 3 3 0 0 3]
 [0 0 3 2 0 3 3 3 2 3 0]
 [0 0 1 3 2 3 0 1 3 2 2]
 [0 0 0 1 3 2 3 1 1 3 3]]


In [8]:
getMax(s)

(11, 8)

In [11]:
u = "CATTTTATATCTGAATGAGTTGGCTAAATGGAAAAAGATGTGTCA" + \
    "CTTCTGCAGGAAAGGAACACACTGAGATATGGTCTTCTGGAAAAG" + \
    "TGGAAACTGTGTGAAAACTCTAGACAGCTGTATCATAATAAACAAACTTTTGC" + \
    "AAAAGGAATTCCACACAATCCACTGTTCCCTTTACATTCTTGCCC"

v = "AAATGAGCTAAATACCTAATAAAAAATGGAAAAAAAAAAAAAAAG" + \
    "ATGTGTCACTTCTGCAGGAAAGGAACACACTGAGATATGGTCTTCTGGAAAAG" + \
    "TGGAAACTGTGTGAAAACTCTAGACAGCTGTATCATAATAAACAA" + \
    "ACTTTTGCAAAAGGAATTCCACACCAAAAAAAAAAAAAAAAAAAA"

print(u)
print(v)

match = {('A','A'):  1, ('A','C'): -1, ('A','G'): -1, ('A','T'): -1,
         ('C','A'): -1, ('C','C'):  1, ('C','G'): -1, ('C','T'): -1,
         ('G','A'): -1, ('G','C'): -1, ('G','G'):  1, ('G','T'): -1,
         ('T','A'): -1, ('T','C'): -1, ('T','G'): -1, ('T','T'):  1}

s, b = LocalAlign(u,v,match,-2)
maxij = numpy.unravel_index(numpy.argmax(s), s.shape)
print(maxij, s[maxij])
uhi, vhi = maxij
ulo, vlo = maxij
while (s[ulo,vlo] > 0):
    back = b[ulo,vlo] 
    if (back & 1):
        ulo -= 1
    if (back & 2):
        vlo -= 1
print(u[ulo:uhi])
print(v[vlo:vhi])

CATTTTATATCTGAATGAGTTGGCTAAATGGAAAAAGATGTGTCACTTCTGCAGGAAAGGAACACACTGAGATATGGTCTTCTGGAAAAGTGGAAACTGTGTGAAAACTCTAGACAGCTGTATCATAATAAACAAACTTTTGCAAAAGGAATTCCACACAATCCACTGTTCCCTTTACATTCTTGCCC
AAATGAGCTAAATACCTAATAAAAAATGGAAAAAAAAAAAAAAAGATGTGTCACTTCTGCAGGAAAGGAACACACTGAGATATGGTCTTCTGGAAAAGTGGAAACTGTGTGAAAACTCTAGACAGCTGTATCATAATAAACAAACTTTTGCAAAAGGAATTCCACACCAAAAAAAAAAAAAAAAAAAA
(159, 167) 128
AAAAAGATGTGTCACTTCTGCAGGAAAGGAACACACTGAGATATGGTCTTCTGGAAAAGTGGAAACTGTGTGAAAACTCTAGACAGCTGTATCATAATAAACAAACTTTTGCAAAAGGAATTCCACAC
AAAAAGATGTGTCACTTCTGCAGGAAAGGAACACACTGAGATATGGTCTTCTGGAAAAGTGGAAACTGTGTGAAAACTCTAGACAGCTGTATCATAATAAACAAACTTTTGCAAAAGGAATTCCACAC
